
# TuIT_DeepRL - DRL Project
![OST](./resources/ost_logo.png)
## MiniF1RL: My pygame 2d racing environment for DRL
Author: Lars Herrmann    
Date: 12.04.2024  
Repository: [Github - miniF1RL](https://github.com/lherrman/miniF1RL)   
Python Version: 3.11

## Introduction
After a lot of experimentaition and consideration of what environment to use. (Playing with e.g. TrackManiaRL, Gymnasium Integrated Environemnts and Unity Environments), i decided to implement my own 2d racing environment. The main reason for this is that i wanted a environment where no pre-trained models are available what motivates me to train my own models. As i started watching formula one this season, i wanted to go with a racing environment. 

## Implementing the environment

### The CarModel
As a starting point, i was able to use a Python class of a simple 2d topdown car model that i implemented for another project about a year ago. The class already had methods to draw a simple 2d box car and it's wheels onto a pygame screen. As it wasn't intended as a racing game, and rather a geometrical model, i had to implement an updated method for updating the cars position and velocity to make it more fun to drive. The original CarModel class can be found in my github repository for the [SlamCar Project](https://github.com/lherrman/slamcar-controller) in the file `base_model.py`.

To make this into a racing environment, there were still some parts missing i startet to implement, starting with a track to drive on.

### The Track
In order to have a track to drive on, i decided to draw a black and white image of a track in photoshop. When initializing the car model, i wrote a python method that uses the 'cv2.findContours' method to find the track boundaries in the image. The track is then represented as two lists of points, one for the inner and one for the outer boundary.

![Track Image](./resources/MakeTrack.png)


Python Code:
```python
    def _get_track_boundaries_from_image(self, image_path) -> dict:
        '''
        Using opencv to read the image and find the contours of the track boundaries.
        The boundaries are represented as list of points. ([x1, y1], [x2, y2], ...)
        '''
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        contours, _ = cv2.findContours(image.astype('uint8'), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        assert len(contours) == 2, "There should be exactly 2 contours in the image"

        # Transform and downsample the contours
        downsample_rate = 10
        inner_boundary = contours[0][:, 0, :][::downsample_rate]
        outer_boundary = contours[1][:, 0, :][::downsample_rate]

        # # Append last point to close the loop
        inner_boundary = np.append(inner_boundary, [inner_boundary[0]], axis=0)
        outer_boundary = np.append(outer_boundary, [outer_boundary[0]], axis=0)

        # Scale the contours to make suitable for the simulation
        inner_boundary, outer_boundary = inner_boundary / 60, outer_boundary / 60

        return {
            'inner': inner_boundary,
            'outer': outer_boundary
        }   
```

#### Track progress
To approximate the progress the car has made on the track, i implemented a method `_calculate_track_progress` that calculates the distance to the next point on the inner boundary. The first point on the inner boundary is set as the starting point. The progress is then calculated as the index of the nearest point divided by the total number of points on the inner boundary. This could be improved by using the distance from the starting point to the nearest point on the inner boundary as the progress.

### The observation space

I wanted to keep the observation space simple, so i decided to use 'lidar sensors', that give the car only a few distance measurements in front of it. I decided to use 3 sensors, one in the middle and one on each side of the car in a 45 degree angle. The sensors are implemented as a method of the CarModel class that returns the distances to the track boundaries. 

The implementation of the `_segment_intersection` and `_raycast` methods was prompted to ChatGPT, which returned an working implementation after 2-3 iterations. The method uses an all python implementation for which the performance isn't optimal, but i guess it's good enough for my purposes. As the raycast method has to itterate over each segment of the boundaries, a more performant implementation would be beneficial.

For easier validation, and because it looks cool, i decided to draw the lidar sensors as lines on the pygame screen.

![Lidar Sensors](./resources/lidars.png)


### The action space

The action space is also very simple. To make the problem even easier for the RL algorithm, i decided to always have the car accelerate, as slowing down isn't necessary in this environment. The car can steer left or right, or go straight. To make it a little more interesting, i added a third action, boost, that allows the car to go faster on the straights.

### The reward function





# Training

In [1]:
# Helper functions


In [10]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
import threading
import torch
import pygame
from minif1env import MiniF1RLEnv
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.env_util import make_vec_env
from torch.utils.tensorboard import SummaryWriter
from stable_baselines3.common.callbacks import CheckpointCallback
import os
import glob

def get_writer(texts: dict):
    writer = SummaryWriter()
    for key, value in texts.items():
        writer.add_text(str(key), str(value))
    return writer

# Use a separate thread to render the environment
# Avoids crashing the pygame window during training
def render(env, close_event):
    while True:
        env.render()
        pygame.time.wait(16) 
        # get events for the window to stay responsive
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                close_event.set()
                return
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_g:
                    env.car_model.switch_render_mode()

        if close_event.is_set():
            pygame.quit()
            return
        
def make_env():
    env = MiniF1RLEnv(render_mode='no')
    return env


def latest_checkpoint_file_path(model_description: str):
  checkpoint_dir = "checkpoints"
  checkpoint_files =  glob.glob(os.path.join(checkpoint_dir, f"*{model_description}*/*.zip"))
  if not checkpoint_files:
    return None
  latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
  return latest_checkpoint


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:


vec_env = make_vec_env(make_env, n_envs=10)

# Create the environment
env = MiniF1RLEnv(render_mode='human')

# Create and start the rendering thread
close_event = threading.Event()
render_thread = threading.Thread(target=render, args=(env,close_event))
render_thread.start()


# Set the training parameters
total_timesteps = 1_000_000
model_description = "PPO_MlpPolicy_64_64_R7"
load_checkpoint = True


latest_checkpoint = latest_checkpoint_file_path(model_description)
if load_checkpoint and latest_checkpoint:
  print(f"Loading checkpoint: {latest_checkpoint}")
  model = PPO.load(latest_checkpoint, env, verbose=1)
else:
  model = PPO("MlpPolicy", env, verbose=1)
run_name = datetime.now().strftime("%Y%m%d_%H%M%S") + "_" + model_description

# Define the policy network and enable GPU usage
policy_kwargs = dict(activation_fn=torch.nn.ReLU, net_arch=[64, 64])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tensorboard writer
writer = get_writer({"run_name": run_name,
                    "model_description": model_description,
                    "total_timesteps": total_timesteps,
                    "env_name": "MiniF1RLEnv",
                    "reward_weights": env.get_reward_weights(),
                    "device": device,
                    "policy_kwargs": policy_kwargs})

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=10_000,
  save_path=f"./checkpoints/{run_name}",
  name_prefix=f"{run_name}_checkpoint",
  save_replay_buffer=True,
  save_vecnormalize=True,
)

# Train the model
model.learn(total_timesteps=total_timesteps, 
            tb_log_name=run_name, 
            callback=checkpoint_callback)
model.save(f"models/{run_name}")

# Close the rendering thread and environment
close_event.set()
render_thread.join()
env.close()

# Close the tensorboard writer
writer.close()



Loading checkpoint: checkpoints\20240507_215905_PPO_MlpPolicy_64_64_R7\20240507_215905_PPO_MlpPolicy_64_64_R7_checkpoint_60000_steps.zip
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.46e+03 |
|    ep_rew_mean     | 1.3e+03  |
| time/              |          |
|    fps             | 132      |
|    iterations      | 1        |
|    time_elapsed    | 15       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.45e+03     |
|    ep_rew_mean          | 1.3e+03      |
| time/                   |              |
|    fps                  | 120          |
|    iterations           | 2            |
|    time_elapsed         | 34           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl  

KeyboardInterrupt: 

# Inference


In [14]:
model_description = "PPO_MlpPolicy_64_64_R7"


env = MiniF1RLEnv(render_mode='human', track_image_path="track2.png")

# Inference

# Load the latest checkpoint
latest_checkpoint = latest_checkpoint_file_path(model_description)
if not latest_checkpoint:
    raise ValueError(f"No checkpoint found for model description: {model_description}")

model = PPO.load(latest_checkpoint)

# Create and start the rendering thread
close_event = threading.Event()
render_thread = threading.Thread(target=render, args=(env,close_event))
render_thread.start()

# Run the model
for i in range(100):
    obs, info = env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info, idk = env.step(action)

        if close_event.is_set():
            break

    # Close the rendering thread and environment


close_event.set()
render_thread.join()
env.close()





# Training Log

## Tests 01
Model:  
Timesteps: 10'000   
Training Time: 1h 10min 
Algorithm: A2C
Result: The car learned to always directly drive into the right border.   
Explaination. After checking the rewards that were given to the model, i realized that the reward function was not working as intended. The weight for the progress reward was way to low so the model did not learn to follow the track. Probebly also the punishment for hitting the track boundaries was to low.

## Tests 02
Model: 
Timesteps: 10'000
Training Time: 1h 10min
Algorithm: PPO
